# Run application

In this page I will introduce how to run `fastapi` applications in general and in particular how to run examples related to `fastapi` in the site.

<a href="https://fastapi.tiangolo.com/deployment/docker/">Here</a> you can find out more details about using `fastapi` in the docker.

## Necessary

## Update programme

The most convenient way to experiment with a container containing `fastapi` is to swap the program on the fly, so you can run many examples through one container. That's why I usually connect the used executing py file as a volume (so that changes on the computer get into the container at once).

But to implement it, you also need to run `uvicorn` with the `--reload` flag, which will make it track changes in the programme and update with it.

So, in the following cells:

- The container stats with a program that sends `initial line` as a response;
- Try in with `curl` - all right, got `initial line`;
- Then change the `reload.py` file to respond with `updated line`;
- Immediately try again with the same `curl` - we've got an `updated line` as response.

In [8]:
%%writefile run_application_files/reload.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "initial line"

Overwriting run_application_files/reload.py


In [9]:
!docker run --rm -itd\
    --name test_container\
    -v ./run_application_files/reload.py:/reload.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload reload:my_first_app

3f1516b767432efb4f23e4b12eeeb13c7d0b0d6fa71f05c59881c305681d9e6a


In [10]:
!curl localhost:8000

"initial line"

In [11]:
%%writefile run_application_files/reload.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "updated line"

Overwriting run_application_files/reload.py


In [12]:
!curl localhost:8000

"updated line"

In [13]:
!docker stop test_container

test_container


##